In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json



X = np.load( 'gdrive/My Drive/Colab Notebooks/data_E/X.npy')
Y1 = np.load( 'gdrive/My Drive/Colab Notebooks/data_E/Y1.npy')
Y2 = np.load( 'gdrive/My Drive/Colab Notebooks/data_E/Y2.npy')



In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train1, y_test1 = train_test_split(X, Y1, test_size=0.3, random_state=10)

In [ ]:
X_train, X_test, y_train2, y_test2 = train_test_split(X, Y2, test_size=0.3, random_state=10)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class MultiTaskLoss(nn.Module):
    def __init__(self, tasks):
        super(MultiTaskLoss, self).__init__()
        self.tasks = nn.ModuleList(tasks)
        self.sigma = nn.Parameter(torch.ones(len(tasks)))
        self.mse = nn.MSELoss()

    def forward(self, x, targets):
       l = [self.mse(f(x), y) for y, f in zip(targets, self.tasks)]
       l = 0.5 * torch.Tensor(l) / self.sigma**2
       l = l.sum() + torch.log(self.sigma.prod())
       return l

f1 = nn.Linear(5, 1, bias=False)
f2 = nn.Linear(5, 1, bias=False)

x = torch.randn(3, 5)
y1 = torch.randn(3)
y2 = torch.randn(3)

mtl = MultiTaskLoss([f1, f2])

print(list(mtl.parameters()))

optimizer = optim.SGD(mtl.parameters(), lr = 0.1)
optimizer.zero_grad()
mtl(x, [y1, y2]).backward()
optimizer.step()


[Parameter containing:
tensor([1., 1.], requires_grad=True), Parameter containing:
tensor([[ 0.0645, -0.3085, -0.0308,  0.0225, -0.3781]], requires_grad=True), Parameter containing:
tensor([[-0.2141, -0.4155,  0.1545,  0.1861,  0.0534]], requires_grad=True)]


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([3])) that is different to the input size (torch.Size([3, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [ ]:
print(list(mtl.parameters()))


[Parameter containing:
tensor([1.1045, 1.0574], requires_grad=True), Parameter containing:
tensor([[ 0.0645, -0.3085, -0.0308,  0.0225, -0.3781]], requires_grad=True), Parameter containing:
tensor([[-0.2141, -0.4155,  0.1545,  0.1861,  0.0534]], requires_grad=True)]


In [ ]:
optimizer.zero_grad()
mtl(x, [y1, y2]).backward()
optimizer.step()
print(list(mtl.parameters()))


[Parameter containing:
tensor([1.2896, 1.1774], requires_grad=True), Parameter containing:
tensor([[ 0.0645, -0.3085, -0.0308,  0.0225, -0.3781]], requires_grad=True), Parameter containing:
tensor([[-0.2141, -0.4155,  0.1545,  0.1861,  0.0534]], requires_grad=True)]


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([3])) that is different to the input size (torch.Size([3, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class MultiTaskLoss(nn.Module):
    def __init__(self, model, loss_fn, eta):
        super(MultiTaskLoss, self).__init__()
        self.model = model
        self.loss_fn = loss_fn
        self.eta = nn.Parameter(torch.Tensor(eta))

    def forward(self, input, targets):
        outputs = self.model(input)
        print("ppp", outputs[1], targets[1],"kkk")

        loss = [l(o,y) for l, o, y in zip(self.loss_fn, outputs, targets)]
        total_loss = torch.Tensor(loss) * torch.exp(-self.eta) + self.eta
        return loss, total_loss.sum() # omit 1/2

class MultiTaskModel(nn.Module):
    def __init__(self):
        super(MultiTaskModel, self).__init__()
        self.e  = nn.Linear(5, 5, bias=False)
        self.f1 = nn.Linear(5, 2, bias=False)
        self.f2 = nn.Linear(5, 3, bias=False)

    def forward(self, input):
        x = self.e(input)
        outputs = [self.f1(x), self.f2(x)]
        return outputs

## For the normal distribution,
loss_fn1 = nn.MSELoss()
## For the Laplace distribution,
# loss_fn1 = nn.L1Loss()
##
## Note the original work uses the L1 loss for Instance Segmentation
## and Depth Regression, as described at page 6.
## https://arxiv.org/abs/1705.07115
##

cel = nn.CrossEntropyLoss()
def loss_fn2(x, cls):
    return 2 * cel(x, cls)

mtl = MultiTaskLoss(model=MultiTaskModel(),
                    loss_fn=[loss_fn1, loss_fn2],
                    eta=[2.0, 1.0])

print(list(mtl.parameters()))

x = torch.randn(3,5)
y1 = torch.randn(3, 2)
y2 = torch.LongTensor([0, 2, 1])

optimizer = optim.SGD(mtl.parameters(), lr=0.1)
optimizer.zero_grad()
loss, total_loss = mtl(x, [y1, y2])
print(loss, total_loss)
total_loss.backward()
optimizer.step()


[Parameter containing:
tensor([2., 1.], requires_grad=True), Parameter containing:
tensor([[-2.8784e-01, -6.9775e-02, -1.1250e-01, -3.5962e-01, -7.9500e-02],
        [-2.6096e-01,  1.5046e-01, -7.1984e-02, -1.3936e-01,  2.6928e-01],
        [-4.2288e-01,  1.7780e-01,  2.9235e-02,  3.6819e-01,  2.3035e-01],
        [ 9.0565e-02,  3.0443e-01, -3.6462e-01, -3.7437e-01,  4.2242e-01],
        [-9.6851e-02, -1.2710e-01, -6.3388e-05,  3.7744e-02,  8.9207e-02]],
       requires_grad=True), Parameter containing:
tensor([[-0.4296,  0.2982,  0.0220,  0.0688,  0.3683],
        [ 0.2635,  0.0044,  0.1972,  0.4462,  0.3725]], requires_grad=True), Parameter containing:
tensor([[ 0.2944, -0.0924,  0.3855,  0.2190, -0.2501],
        [-0.0858,  0.4134,  0.4399,  0.3186,  0.1329],
        [ 0.2883, -0.2810, -0.2576, -0.2514, -0.3879]], requires_grad=True)]
ppp tensor([-0.0089,  0.0015,  0.0310], grad_fn=<SqueezeBackward3>) tensor([0, 2, 1]) kkk


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([3, 2])) that is different to the input size (torch.Size([2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: ignored

In [ ]:
print(list(mtl.parameters()))

[Parameter containing:
tensor([1.9124, 0.9825], requires_grad=True), Parameter containing:
tensor([[ 0.1035, -0.0860, -0.2420,  0.3506, -0.4184],
        [ 0.0379,  0.0590, -0.3528,  0.1767, -0.0258],
        [ 0.0195,  0.0472, -0.0838, -0.2745,  0.3766],
        [ 0.3776,  0.1842, -0.1624, -0.1965,  0.0585],
        [ 0.1905,  0.3351,  0.3259,  0.0418,  0.3488]], requires_grad=True), Parameter containing:
tensor([[-0.1931,  0.2829, -0.0435,  0.3936,  0.1867],
        [-0.0602, -0.3225, -0.4322,  0.1304, -0.2328]], requires_grad=True), Parameter containing:
tensor([[-0.3826, -0.0905, -0.3647,  0.2619, -0.3270],
        [ 0.3688, -0.4137,  0.2477,  0.0929, -0.0612],
        [-0.1673,  0.2014,  0.4349,  0.1554, -0.0880]], requires_grad=True)]


In [ ]:
print(loss, total_loss)


[tensor(0.9183, grad_fn=<MseLossBackward0>), tensor(2.2429, grad_fn=<MulBackward0>)] tensor(3.9494, grad_fn=<SumBackward0>)


In [ ]:
optimizer.zero_grad()
loss, total_loss = mtl(x, [y1, y2])
print(loss, total_loss)
total_loss.backward()
optimizer.step()


[tensor(0.9183, grad_fn=<MseLossBackward0>), tensor(2.2429, grad_fn=<MulBackward0>)] tensor(3.8703, grad_fn=<SumBackward0>)


In [ ]:
optimizer.zero_grad()
loss, total_loss = mtl(x, [y1, y2])
print(loss, total_loss)
total_loss.backward()
optimizer.step()


[tensor(0.9183, grad_fn=<MseLossBackward0>), tensor(2.2429, grad_fn=<MulBackward0>)] tensor(2.8748, grad_fn=<SumBackward0>)


In [ ]:
print(list(mtl.parameters()))


[Parameter containing:
tensor([0.4764, 0.8286], requires_grad=True), Parameter containing:
tensor([[ 0.1035, -0.0860, -0.2420,  0.3506, -0.4184],
        [ 0.0379,  0.0590, -0.3528,  0.1767, -0.0258],
        [ 0.0195,  0.0472, -0.0838, -0.2745,  0.3766],
        [ 0.3776,  0.1842, -0.1624, -0.1965,  0.0585],
        [ 0.1905,  0.3351,  0.3259,  0.0418,  0.3488]], requires_grad=True), Parameter containing:
tensor([[-0.1931,  0.2829, -0.0435,  0.3936,  0.1867],
        [-0.0602, -0.3225, -0.4322,  0.1304, -0.2328]], requires_grad=True), Parameter containing:
tensor([[-0.3826, -0.0905, -0.3647,  0.2619, -0.3270],
        [ 0.3688, -0.4137,  0.2477,  0.0929, -0.0612],
        [-0.1673,  0.2014,  0.4349,  0.1554, -0.0880]], requires_grad=True)]
